# <center> Drug prediction

# Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

# Importing the data

In [ ]:
data = pd.read_csv('../input/drug-classification/drug200.csv')

In [ ]:
data

In [ ]:
data.info()

# EDA

In [ ]:
data.groupby('Drug').median()

In [ ]:
sns.countplot(data['Drug'])

In [ ]:
plt.figure(figsize=(20,12))
plt.subplot(2,2,1)
sns.countplot('Sex',hue='Drug',data=data)
plt.subplot(2,2,2)
sns.countplot('Cholesterol',hue='Drug',data=data)
plt.subplot(2,2,3)
sns.countplot('BP',hue='Drug',data=data)

In [ ]:
plt.figure(figsize=(20,12))
plt.subplot(2,2,1)
sns.boxplot(y='Age',x='Drug',data=data)
plt.subplot(2,2,2)
sns.boxplot(y='Na_to_K',x='Drug',data=data)

# One hot encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
to_be_encoded = ['Sex','BP','Cholesterol']
label_encoder = LabelEncoder()
dfs = []
for i in to_be_encoded:
    temp = pd.DataFrame({'Before Encoding':data[i].unique(),'After Encoding':label_encoder.fit_transform(data[i].unique())})
    #print(temp.sort_values(by=['After Encoding'],),'\n')
    dfs.append([temp.sort_values(by=['After Encoding'],),i])
    data[i] = label_encoder.fit_transform(data[i])
#temp.sort_values(by=['After Encoding'])

In [ ]:
print(dfs[0][1])
dfs[0][0]

In [ ]:
print(dfs[1][1])
dfs[1][0]

In [ ]:
print(dfs[2][1])
dfs[2][0]

# Feature Selection

In [ ]:
X = data.drop('Drug',axis=1)
Y = data['Drug']

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(X, Y)
per = []
for i in fs.scores_:
    per.append(round(((i/sum(fs.scores_))*100),3))

features_data = pd.DataFrame({'Feature':X.columns,'Scores':fs.scores_,'Importance (%)':per}).sort_values(by=['Scores'],ascending=False)

plt.figure(figsize=(9,3))
sns.barplot( 'Importance (%)','Feature',orient='h',data=features_data)
insignificant = features_data.loc[features_data['Importance (%)']<0.005]['Feature'].unique()

In [ ]:
features_data

# test train split

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=100)

# Model Selection

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

#XGB
import xgboost as xgb
from xgboost import XGBClassifier
xgb = XGBClassifier() 


# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

#RFC
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

#KNN
from sklearn.neighbors import KNeighborsClassifier
accuracy = []
for i in range(1,40):    
    kn = KNeighborsClassifier(n_neighbors=i)
    kn.fit(X_train,Y_train)
    predK = kn.predict(X_test)
    accuracy.append([accuracy_score(Y_test,predK),i])
    #print('Tested for k =',i)
temp = accuracy[0]
for m in accuracy:
    if temp[0] < m[0]:
        temp=m
knn = KNeighborsClassifier(n_neighbors=temp[1])

#SVM
from sklearn.svm import SVC
svc = SVC()

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.1,1, 10, 100, 1000,2000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} 
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)

print('Models Imported')

In [ ]:
model_acc = []
models = [xgb,lr,rfc,knn,svc,grid]
#model_name = ['xgb','lr','rfc','kno','svc','grid']
for i in models:
    i.fit(X_train,Y_train)
    model_acc.append(accuracy_score(Y_test,i.predict(X_test)))
                      
models = pd.DataFrame({'Models':models,'Accuracy':model_acc})

In [ ]:
models = models.sort_values(by=['Accuracy'],ascending=False).reset_index().drop('index',axis=1)
best = models['Models'][0]
models['Models']=models['Models'].astype(str).str.split("(", n = 2, expand = True)[0]
models

In [ ]:
print('Hence the best model is',models['Models'][0],'with an accuracy of',round((models['Accuracy'][0]*100),2),'%')
print('\nThe classification report is:')
print(classification_report(Y_test,best.predict(X_test)))